In [12]:
from IPython.utils import io
import csv
import numpy as np
from sklearn.model_selection import cross_val_predict, cross_val_score
with io.capture_output() as captured:  
   #%run ../../2-FeatureExtraction/BowTF/BowTF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
   %run ../../2-FeatureExtraction/BagOfWords/BagOfWords.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features
   #%run ../../2-FeatureExtraction/TF-IDF/TF-IDF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score, classification_report
from imblearn.over_sampling import SMOTE

#### Naive Bayes Training

In [14]:
alpha = 0.05
fit_prior = False
Oversampling = True
CrossValidation = False
model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
ModelInfo = {
   "Model": "NB-2",
   "Alpha": alpha,
   "Fit Prior": fit_prior,
   "Oversampling": Oversampling,
   "CrossValidation": CrossValidation,
}

if Oversampling:
   sm = SMOTE(random_state=42, k_neighbors=8)
   x, y2 = sm.fit_resample(x, y2)

#### Naive Bayes Prediction

In [15]:
if CrossValidation:
   X_c = np.concatenate((x.A, X.A), axis=0)
   Y_c2 = np.concatenate((y2, Y2), axis=0)
   outputs = cross_val_predict(model, X_c, Y_c2, cv=5)
   Report = classification_report(Y_c2, outputs, output_dict=True)
   print(classification_report(Y_c2, outputs))
   F1 = f1_score(Y_c2, outputs, average='macro')
   print('F1 score: ', F1)

else:
   model = model.fit(x, y2)
   outputs = model.predict(X)
   Report = classification_report(Y2, outputs, output_dict=True)
   print(classification_report(Y2, outputs))
   F1 = f1_score(Y2, outputs, average='macro')
   print('F1 score: ', F1)

              precision    recall  f1-score   support

           0       0.28      0.50      0.36        10
           1       0.85      0.80      0.83       145
           2       0.73      0.61      0.66       545
           3       0.13      0.12      0.12        17
           4       0.47      0.62      0.53       128
           5       0.23      0.39      0.29        82
           6       0.23      0.25      0.24        20
           7       1.00      0.50      0.67         2
           8       0.00      0.00      0.00        15
           9       0.41      0.36      0.38        36

    accuracy                           0.58      1000
   macro avg       0.43      0.41      0.41      1000
weighted avg       0.62      0.58      0.60      1000

F1 score:  0.4072581296582761


#### Log Model Info

In [23]:
real_outputs = model.predict(X_test)
Dicct =  {'advice': 0, 'celebrity':1, 'info_news':2, 'others':3, 'personal':4, 'plan':5, 'requests':6, 'restrictions':7, 'rumors':8, 'unrelated':9}
# invert the dictionary
Dicct = {v: k for k, v in Dicct.items()}
# save it as CSV
with open('NB-2.csv', 'w', newline='') as file:
   writer = csv.writer(file)
   writer.writerow(["Id", "Category"])
   for i in range(len(real_outputs)):
      writer.writerow([i, Dicct[real_outputs[i]]])

In [17]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)